In [2]:
# Initialize Dependencies
import boto3
import json
import numpy as np
import pandas as pd

In [ ]:
# Initialize clients
client = boto3.client('rekognition')
#Make a datafram by first creating a list of the photos
s3_resource = boto3.resource('s3')
my_bucket = s3_resource.Bucket('qtm350-final-project')
summaries = my_bucket.objects.all()
image_names = [image.key for image  in summaries]

In [47]:
#Reviewing what data we have in our s3 bucket
!aws s3 ls final-project-qtm350

2022-11-24 20:51:42    3389327 guan1.jpeg
2022-11-24 20:51:43    3748740 guan2.jpeg
2022-11-24 20:51:44    4531495 guan3.jpeg
2022-11-24 20:51:43    4411550 guan4.jpeg
2022-11-24 21:25:44    2099947 hsu1.jpeg
2022-11-24 21:25:45    2082578 hsu2.jpeg
2022-11-24 21:25:45    2094801 hsu3.jpeg
2022-11-24 21:25:47    2142550 hsu4.jpeg
2022-11-24 21:25:46     193429 luo1.jpeg
2022-11-24 21:25:46     198693 luo2.jpeg
2022-11-24 21:25:44     213083 luo3.jpeg
2022-11-24 21:25:47     189164 luo4.jpeg
2022-11-24 21:25:47     508724 meng1.jpg
2022-11-24 21:25:48     479926 meng2.jpg
2022-11-24 21:25:48     465316 meng3.jpg
2022-11-24 21:25:48     464254 meng4.jpg
2022-11-24 21:25:49    1843220 peng1.jpeg
2022-11-24 21:25:49    1770896 peng2.jpeg
2022-11-24 21:25:50    2031102 peng3.jpeg
2022-11-24 21:25:50    1947570 peng4.jpeg
2022-11-24 21:25:51     370230 yuan1.jpeg
2022-11-24 21:25:51     435862 yuan2.jpeg
2022-11-24 21:25:52     470192 yuan3.jpeg
2022-11-24 21:25:52     422453 yuan4.jpeg


Define a python function that return the smile value of a given photo, that is, whether Rekognition thinks the face in the photo is smiling. 

In [6]:
def smile_analysis(photo):

    response = client.detect_faces(Image={'S3Object':{'Bucket':'qtm350-final-project','Name':photo}},Attributes=['ALL'])
 
    for faceDetail in response['FaceDetails']:
        return faceDetail['Smile']['Value']

Define a python function that return the Eyeglasses value of a given photo, that is, whether Rekognition thinks the face in the photo is wearing glasses.

In [7]:
def eyeglasses_analysis(photo):
    
    response = client.detect_faces(Image={'S3Object':{'Bucket':'qtm350-final-project','Name':photo}},Attributes=['ALL'])
        
    for faceDetail in response['FaceDetails']:
        return faceDetail['Eyeglasses']['Value']

Define a python function that return the Emotions type of a given photo, that is, what emotion Rekognition thinks the face in the photo is showing.

In [8]:
def emotion_analysis(photo):
    
    response = client.detect_faces(Image={'S3Object':{'Bucket':'qtm350-final-project','Name':photo}},Attributes=['ALL'])
        
    for faceDetail in response['FaceDetails']:
        return faceDetail['Emotions'][0]['Type']

Create a dataframe called df_facial_analysis that demonstrate the Smile value, Eyeglasses value, and Emotions type of each photo.  

In [9]:
df_facial_analysis = pd.DataFrame({'Name':image_names})
df_facial_analysis['Smile'] = [smile_analysis(photo) for photo in df_facial_analysis['Name']]
df_facial_analysis['Glasses'] = [eyeglasses_analysis(photo) for photo in df_facial_analysis['Name']]
df_facial_analysis['Emotion'] = [emotion_analysis(photo) for photo in df_facial_analysis['Name']]
df_facial_analysis

,Name,Smile,Glasses,Emotion
0,guan1.jpeg,False,False,CALM
1,guan2.jpeg,False,True,CALM
2,guan3.jpeg,False,True,FEAR
3,guan4.jpeg,False,True,FEAR
4,hsu1.jpeg,False,False,CALM
5,hsu2.jpeg,False,True,CALM
6,hsu3.jpeg,False,True,FEAR
7,hsu4.jpeg,False,True,SAD
8,luo1.jpeg,False,False,CALM
9,luo2.jpeg,False,True,CALM


Define a python function that return the Similarity value of a given photo, that is, whether Rekognition thinks the face in the photo is smiling.

In [ ]:
 def extract_similarity(sourcephoto, targetphoto):
    try:
        comparison = client.compare_faces(
            SourceImage= {'S3Object':{'Bucket':'qtm350-final-project', 'Name':sourcephoto}},
            TargetImage = {'S3Object':{'Bucket':'qtm350-final-project','Name':targetphoto}})
        similarity = comparison['FaceMatches'][0]['Similarity']
    except Exception:
        similarity = np.nan
    return similarity

Create a dataframe called df_similarity that demonstrate the similarities between 

In [11]:
df_similarity = pd.DataFrame({'Name':image_names})
df_similarity['Similarity-guan'] = [extract_similarity('guan1.jpeg', photo) for photo in df_similarity['Name']]
df_similarity['Similarity-hsu'] = [extract_similarity('hsu1.jpeg', photo) for photo in df_similarity['Name']]
df_similarity['Similarity-luo'] = [extract_similarity('luo1.jpeg', photo) for photo in df_similarity['Name']]
df_similarity['Similarity-meng'] = [extract_similarity('meng1.jpg', photo) for photo in df_similarity['Name']]
df_similarity['Similarity-peng'] = [extract_similarity('peng1.jpeg', photo) for photo in df_similarity['Name']]
df_similarity['Similarity-yuan'] = [extract_similarity('yuan1.jpeg', photo) for photo in df_similarity['Name']]
df_similarity

,Name,Similarity-guan,Similarity-hsu,Similarity-luo,Similarity-meng,Similarity-peng,Similarity-yuan
0,guan1.jpeg,100.000000,NaN,NaN,NaN,NaN,NaN
1,guan2.jpeg,99.999062,NaN,NaN,NaN,NaN,NaN
2,guan3.jpeg,99.693840,NaN,NaN,NaN,NaN,NaN
3,guan4.jpeg,92.780724,NaN,NaN,NaN,NaN,NaN
4,hsu1.jpeg,NaN,100.000000,NaN,NaN,NaN,NaN
5,hsu2.jpeg,NaN,99.999649,NaN,NaN,NaN,NaN
6,hsu3.jpeg,NaN,99.970329,NaN,NaN,NaN,NaN
7,hsu4.jpeg,NaN,99.691322,NaN,NaN,NaN,NaN
8,luo1.jpeg,NaN,NaN,100.000000,NaN,NaN,NaN
9,luo2.jpeg,NaN,NaN,99.999672,NaN,NaN,NaN
